In [1]:
!nvidia-smi

Fri Dec  4 21:00:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   25C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
import warnings

warnings.filterwarnings('ignore')
plt.style.use('dark_background')

In [3]:
import pandas as pd
import os

path = './'
mask = 0
no_mask = 1

mask_path = '/home/lab07/test_data_3lass/with_mask'
no_mask_path = '/home/lab07/test_data_3lass/without_mask'
inc_mask_path = '/home/lab07/test_data_3lass/incorrect_mask'


file = []
for filename in os.listdir(mask_path ):
    file.append(mask_path + '/' + filename)

mask_df = pd.DataFrame(columns = ['file','label'])
mask_df['file'] = file
mask_df['label'] = 0
# print(mask_df[:3])

file = []
for filename in os.listdir(no_mask_path ):
    file.append(no_mask_path + '/' + filename)

no_mask_df = pd.DataFrame(columns = ['file','label'])
no_mask_df['file'] = file
no_mask_df['label'] = 1
# print(no_mask_df[:3])

file = []
for filename in os.listdir(inc_mask_path):
    file.append(inc_mask_path + '/' + filename)

inc_mask_df = pd.DataFrame(columns = ['file','label'])
inc_mask_df['file'] = file
inc_mask_df['label'] = 2


df = pd.concat([mask_df, no_mask_df,inc_mask_df], axis=0)
# train_df, valid_df = train_test_split(df, test_size=0.2, random_state = 25, stratify = df['label'])
# train_df, test_df = train_test_split(train_df, test_size=0.2, random_state = 25, stratify = train_df['label'])

df_shuffled=df.sample(frac=1).reset_index(drop=True)
# df.to_csv("/home/lab07/test_data_3lass/test.csv")
# df_shuffled.to_csv("/home/lab07/test_data_3lass/test_shuffle.csv")

print(df)
print(df['label'].value_counts())

print(df_shuffled)
print(df_shuffled['label'].value_counts())




                                                 file  label
0   /home/lab07/test_data_3lass/with_mask/MOXA_749...      0
1   /home/lab07/test_data_3lass/with_mask/MOXA_401...      0
2   /home/lab07/test_data_3lass/with_mask/MOXA_509...      0
3   /home/lab07/test_data_3lass/with_mask/MOXA_690...      0
4   /home/lab07/test_data_3lass/with_mask/MOXA_907...      0
..                                                ...    ...
45  /home/lab07/test_data_3lass/incorrect_mask/41.jpg      2
46  /home/lab07/test_data_3lass/incorrect_mask/21.jpg      2
47  /home/lab07/test_data_3lass/incorrect_mask/48.jpg      2
48   /home/lab07/test_data_3lass/incorrect_mask/4.jpg      2
49   /home/lab07/test_data_3lass/incorrect_mask/1.jpg      2

[150 rows x 2 columns]
2    50
1    50
0    50
Name: label, dtype: int64
                                                  file  label
0     /home/lab07/test_data_3lass/incorrect_mask/9.jpg      2
1    /home/lab07/test_data_3lass/incorrect_mask/10.jpg      2
2    /ho

In [4]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [18]:
# facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
with tf.device('/device:GPU:0'):
    
#     model = load_model('/home/lab07/Model_predict/Xception_color_soft.h5')
#     model = load_model('/home/lab07/Model_predict/MobileNetV2_soft.h5')
    model = load_model('/home/lab07/Model_predict/ResNet152V2_soft.h5')
#     model = load_model('/home/lab07/Model_predict/NASNetMobile_soft.h5')
#     model = load_model('/home/lab07/Model_predict/test_mask_model_Densenet201_color_3class.h5')
#     model2 = load_model('/home/lab07/Model_predict/test_mask_model_Xeption_color_pre.h5')


In [6]:
def make_crop_img(image_file, detector):
    result = detector.detect_faces(image_file)
    if result == []:
        print('No information')
        return []
    else:
        bounding_box = result[0]['box']
        
        for a in range(4):
            if bounding_box[a] < 0:
                bounding_box[a] = 0
                
        resizeImageNDArray = image_file.copy()
        resizeImageNDArray = image_file[bounding_box[1]:bounding_box[1]+bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]

        nose = nose_cascade.detectMultiScale(resizeImageNDArray)
        # print('코 인식좌표:', nose)
        if type(nose) == type(np.array([])):
            # 코가 검출되었다면 코 위치에 대한 좌표정보를 리턴받는다
#             nx=nose[0][0]; ny=nose[0][1]; nw=nose[0][2]; nh=nose[0][3]
#             nose_center = (x + nx + nw//2, y + ny + nh//2)
#             print('코 인식좌표:', nose_center)
            nd = 'detected_nose' # 코 검출 : 마스크 불량착용

        else:
            nd = 'not_detected_nose' # 코 미검출 : 코스크는 아님
        
        return resizeImageNDArray, nd
    
haarcascade_dir = '/home/lab07/MTCNN HaarCascade'
nose_cascade = cv2.CascadeClassifier(haarcascade_dir + '/haarcascade_mcs_nose.xml')
detector = MTCNN()

In [ ]:
# test = cv2.cvtColor(cv2.imread('/home/lab07/test_data_3lass/with_mask/MOXA_1124.jpg'), cv2.COLOR_BGR2RGB)

test = cv2.cvtColor(cv2.imread('/home/lab07/test_data_3lass/incorrect_mask/32.jpg'), cv2.COLOR_BGR2RGB) 

image, message = make_crop_img(test, detector)
face_input = cv2.resize(image, dsize=(224, 224))

face_input = preprocess_input(face_input)
message = classify_nose(face_input)
plt.imshow(face_input)

face_input = np.expand_dims(face_input, axis=0)
mask1 = model.predict(face_input)
result = np.argmax(mask1)
# message = classify_nose(test)
print(message)
print(result)
# plt.imshow(test)




In [19]:
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from mtcnn import MTCNN

# df_shuffled

plt.style.use('dark_background')
with tf.device('/device:GPU:0'):
    # df = pd.read_csv('/home/lab07/test_data_3lass/test_shuffle.csv')
    # print(df)
    ddff = []
    pd = []
    for t,i in enumerate(df_shuffled['file']):
        try:
            images = cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB) 
#             message = classify_nose(images)
            image, message = make_crop_img(images, detector)

            face_input = cv2.resize(image, dsize=(224, 224))
        #     face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
            face_input = preprocess_input(face_input)
            face_input = np.expand_dims(face_input, axis=0)
            mask1 = model.predict(face_input)
            result = np.argmax(mask1)

            if result == 0:
                pd.append(0)
                ddff.append(0)
#                 if message == 'detected_nose':
#                     ddff.append(2)
#                     print(message)
#                 else:
#                     ddff.append(0)

            elif result == 1:
                pd.append(1)
                if message == 'not_detected_nose':
                    ddff.append(0)
                else:
                    ddff.append(1)

            else:
                pd.append(2)
                if message == 'not_detected_nose':
                    ddff.append(0)
                else:
                    ddff.append(2)
        except:
            ddff.append(4)
            pd.append(4)
        
df_shuffled['result'] = ddff
df_shuffled['predict'] = pd
# display(ddff)
display(df_shuffled)            
# display(df)        
# df.to_csv("/home/lab07/test_data_3lass/result_ex.csv")   

No information


,file,label,result,predict
0,/home/lab07/test_data_3lass/incorrect_mask/9.jpg,2,1,1
1,/home/lab07/test_data_3lass/incorrect_mask/10.jpg,2,1,1
2,/home/lab07/test_data_3lass/incorrect_mask/22.jpg,2,1,1
3,/home/lab07/test_data_3lass/incorrect_mask/46.jpg,2,2,2
4,/home/lab07/test_data_3lass/incorrect_mask/40.jpg,2,1,1
...,...,...,...,...
145,/home/lab07/test_data_3lass/incorrect_mask/18.jpg,2,2,2
146,/home/lab07/test_data_3lass/with_mask/MOXA_690...,0,0,0
147,/home/lab07/test_data_3lass/incorrect_mask/45.jpg,2,0,2
148,/home/lab07/test_data_3lass/without_mask/MOXA_...,1,1,1


# 점수 확인

In [20]:
score = 0

for l,item in df_shuffled.iterrows():
    if item['label'] == item['result']:
        score += 1
    else:
        pass

per = (score / df_shuffled.shape[0]) *100
print(per)

77.33333333333333


# 잘못된 마스크, 미착용을 같은 범주에 포함시킬 때

In [21]:
score = 0
pred = 0

for l,item in df_shuffled.iterrows():
    if item['label'] == item['result']:
        score += 1
        
    elif item['label'] == 1 or item['label'] == 2:
        if item['result'] == 1 or item['result'] == 2:
            score += 1
        else:
            pass
    else:
        pass
    
    if item['label'] == item['predict']:
        pred += 1
    elif item['label'] == 1 or item['label'] == 2:
        if item['predict'] == 1 or item['predict'] == 2:
            pred += 1
        else:
            pass
        
    if item['label'] != item['result']:
        print([item['file'],item['label'],item['result'],item['predict']])

# harr 적용시킨 후 1,2 포함
per = (score / df_shuffled.shape[0]) *100
print(per)

# model predict의 1,2 포함
predscore = ( pred / df_shuffled.shape[0]) *100
print(predscore)


# ['/home/lab07/test_data_3lass/incorrect_mask/25.jpg', 2, 0, 1]

['/home/lab07/test_data_3lass/incorrect_mask/9.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/10.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/22.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/40.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/29.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/44.jpg', 2, 0, 0]
['/home/lab07/test_data_3lass/incorrect_mask/13.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/7.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/30.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/12.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/36.jpg', 2, 4, 4]
['/home/lab07/test_data_3lass/incorrect_mask/31.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/41.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/8.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/39.jpg', 2, 1, 1]
['/home/lab07/test_data_3lass/incorrect_mask/11.jpg', 2, 1


# Score 기록

## Xception : 80% ,  93% => harr 91%

## DenseNet201 : 82%  , 94.7% => harr 92%

## MobileNetV2 : 72% , 92.7% => harr 92.7%

## NASNetMobile : 78.0% , 90% => harr 88%

## ResNet152V2 : 77.3% 92.7% => harr 92.7%
